In [1]:
# HF_DATASETS_CACHE='/mnt/e/models/huggingface.cache'
# HF_HOME='/mnt/e/models/huggingface.cache'
import torch
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain_community.llms       import LlamaCpp, CTransformers
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader, WebBaseLoader, PyPDFDirectoryLoader, CSVLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.prompts       import PromptTemplate
from langchain.chains        import LLMChain, RetrievalQA
#1m 1.5s

In [2]:
data = load_dataset("HuggingFaceTB/cosmopedia", "stories", split="train[:100]")
num_examples = len(data)

# data = data.to_pandas()
# data.to_csv("E:/models/huggingface.cache/HuggingFaceTB___cosmopedia/stories/dataset.csv")
print(num_examples)
#3.4s

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/43 [00:00<?, ?it/s]

100


In [3]:
loader = CSVLoader(file_path='/mnt/e/models/huggingface.cache/HuggingFaceTB___cosmopedia/stories/dataset5100.csv')
data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(data)

In [7]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
# model_kwargs = {'device':'cpu'}
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
 model_name=modelPath, 
 model_kwargs=model_kwargs, 
 encode_kwargs=encode_kwargs 
)

In [8]:
db = FAISS.from_documents(docs, embeddings)
# db.save_local("E:/models/huggingface.cache/HuggingFaceTB___cosmopedia/stories/faiss_index_constitution")
# vectorstore.save_local("faiss_index_constitution")

#     # Load from local storage
#     persisted_vectorstore = FAISS.load_local("faiss_index_constitution", embeddings)

#     # Use RetrievalQA chain for orchestration
#     qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
#     result = qa.run(query)
#     print(result)
# 1m 12.2s

In [10]:
model_name = "/mnt/e/models/2b-gemma"
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
#2m 35.3s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    return_tensors='pt',
    max_length=512,
    max_new_tokens=512,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda"
)

In [13]:
llm = HuggingFacePipeline(
 pipeline=pipe,
 model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [14]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [17]:
qa.invoke("Write a story.")

Both `max_new_tokens` (=512) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'query': 'Write a story.',
 'result': "\n\nSure, here is a story written based on the text snippet:\n\nIn a quaint subreddit where stories flourished, I stumbled upon a thread that captured my attention. It was a tale of a young writer who faced adversity and emerged as a champion. The story resonated deeply with me, as it reminded me of the importance of perseverance and the power of storytelling.\n\nThe writer's journey was marked by countless obstacles, including financial struggles and rejection from publishers. Yet, through sheer determination and unwavering belief in his craft, he overcame these challenges. His story served as a reminder that even in the face of adversity, the human spirit can triumph.\n\nThe writer's triumph inspired a community of readers, who shared their own stories of resilience and triumph. Together, we formed a bond of shared experiences, realizing that stories serve as a source of solace, inspiration, and a reminder that we are not alone in our journeys.